In [ ]:
import chipwhisperer as cw
proj = cw.create_project("projects/manual_capture_AES.cwp", overwrite=True)

In [ ]:
from tqdm.notebook import trange
from Crypto.Cipher import AES
import random

N = 5000
ktp = cw.ktp.Basic()
ktp.fixed_key = False

for i in range(random.randint(1, 10)):
    key = ktp.next_key()
text = ktp.next_text()

cipher = AES.new(bytes(key), AES.MODE_ECB)

target.fpga_write(target.REG_CRYPT_KEY, key[::-1])

for i in trange(N, desc='Capturing Traces from the Artix-FPGA'):
    scope.arm()
    
    target.fpga_write(target.REG_CRYPT_TEXTIN, text[::-1])

    target.fpga_write(target.REG_USER_LED, [0x01])
    #trigger the encryption
    target.usb_trigger_toggle()

    ret = scope.capture()
    if ret:
        print("Timeout occured boubouna")
        continue
    output = target.fpga_read(target.REG_CRYPT_CIPHEROUT, 16)
    output = output[::-1]
    assert (list(output) == list(cipher.encrypt(bytes(text)))), "Incorrect encryption result!\nGot {}\nExp {}".format(bytearray(output), bytearray(cipher.encrypt(bytes(text))))
    wave = scope.get_last_trace()
    trace = cw.Trace(wave, text, output, key)
    proj.traces.append(trace)
    text = ktp.next_text()
#encryption finished turn of the LED
target.fpga_write(target.REG_USER_LED, [0])
proj.save()